# Childhood Autistic Spectrum Disorder Screening
### Description
In this project, we're going to take advantage of neural network using Keras API to diagnose Autistic Spectrum Disorder based on behavioral features and individual characteristics. 
### Dataset
This project use a dataset provided by the UCI Machine Learning Repository, which contains screening data for 292 patients.

In [1]:
import sys 
import pandas as pd
import numpy as np
import sklearn
import keras

print("Python: {}".format(sys.version))
print("pandas: {}".format(pd.__version__))
print("numpy: {}".format(np.__version__))
print("sklearn: {}".format(sklearn.__version__))
print("keras: {}".format(keras.__version__))

Using TensorFlow backend.


Python: 3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]
pandas: 0.24.2
numpy: 1.16.2
sklearn: 0.20.3
keras: 2.2.4


### Load the dataset

In [2]:
# read the dataset
from scipy.io import arff
data = arff.loadarff("./data/Autism-Child-Data.arff")

df = pd.DataFrame(data[0])

for col in df.columns:
    if df[col].dtypes != float:
        df[col] = df[col].apply(lambda x: str(x, 'utf-8')) # convert byte object to type string
        df[col] = pd.to_numeric(df[col], errors='ignore') # convert a number in string type to numeric

df['age'] = df['age'].apply(lambda x: str(x)) # convert column age to string type

In [3]:
# examine the dataset
print("Shape of the dataframe: {}".format(df.shape))
print(df.loc[0])

Shape of the dataframe: (292, 21)
A1_Score                      1
A2_Score                      1
A3_Score                      0
A4_Score                      0
A5_Score                      1
A6_Score                      1
A7_Score                      0
A8_Score                      1
A9_Score                      0
A10_Score                     0
age                         6.0
gender                        m
ethnicity                Others
jundice                      no
austim                       no
contry_of_res            Jordan
used_app_before              no
result                        5
age_desc           '4-11 years'
relation                 Parent
Class/ASD                    NO
Name: 0, dtype: object


In [4]:
print(df.dtypes)

A1_Score             int64
A2_Score             int64
A3_Score             int64
A4_Score             int64
A5_Score             int64
A6_Score             int64
A7_Score             int64
A8_Score             int64
A9_Score             int64
A10_Score            int64
age                 object
gender              object
ethnicity           object
jundice             object
austim              object
contry_of_res       object
used_app_before     object
result             float64
age_desc            object
relation            object
Class/ASD           object
dtype: object


In [5]:
# examine the first ten patients
df.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,...,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,result,age_desc,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,...,m,Others,no,no,Jordan,no,5.0,'4-11 years',Parent,NO
1,1,1,0,0,1,1,0,1,0,0,...,m,'Middle Eastern ',no,no,Jordan,no,5.0,'4-11 years',Parent,NO
2,1,1,0,0,0,1,1,1,0,0,...,m,?,no,no,Jordan,yes,5.0,'4-11 years',?,NO
3,0,1,0,0,1,1,0,0,0,1,...,f,?,yes,no,Jordan,no,4.0,'4-11 years',?,NO
4,1,1,1,1,1,1,1,1,1,1,...,m,Others,yes,no,'United States',no,10.0,'4-11 years',Parent,YES
5,0,0,1,0,1,1,0,1,0,1,...,m,?,no,yes,Egypt,no,5.0,'4-11 years',?,NO
6,1,0,1,1,1,1,0,1,0,1,...,m,White-European,no,no,'United Kingdom',no,7.0,'4-11 years',Parent,YES
7,1,1,1,1,1,1,1,1,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,8.0,'4-11 years',Parent,YES
8,1,1,1,1,1,1,1,0,0,0,...,f,'Middle Eastern ',no,no,Bahrain,no,7.0,'4-11 years',Parent,YES
9,0,0,1,1,1,0,1,1,0,0,...,f,?,no,yes,Austria,no,5.0,'4-11 years',?,NO


In [6]:
# describe the dataset
df.describe()

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,result
count,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000,292.000000
mean,0.633562,0.534247,0.743151,0.551370,0.743151,0.712329,0.606164,0.496575,0.493151,0.726027,6.239726
std,0.482658,0.499682,0.437646,0.498208,0.437646,0.453454,0.489438,0.500847,0.500811,0.446761,2.284882
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,6.000000
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000


### Preprocessing
Step 1: Drop some unnecessary columns

Step 2: Convert string data to categorical labels

In [7]:
# drop unwanted cols
df = df.drop(['result', 'age_desc'], axis=1)

In [8]:
df.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,relation,Class/ASD
0,1,1,0,0,1,1,0,1,0,0,6.0,m,Others,no,no,Jordan,no,Parent,NO
1,1,1,0,0,1,1,0,1,0,0,6.0,m,'Middle Eastern ',no,no,Jordan,no,Parent,NO
2,1,1,0,0,0,1,1,1,0,0,6.0,m,?,no,no,Jordan,yes,?,NO
3,0,1,0,0,1,1,0,0,0,1,5.0,f,?,yes,no,Jordan,no,?,NO
4,1,1,1,1,1,1,1,1,1,1,5.0,m,Others,yes,no,'United States',no,Parent,YES
5,0,0,1,0,1,1,0,1,0,1,4.0,m,?,no,yes,Egypt,no,?,NO
6,1,0,1,1,1,1,0,1,0,1,5.0,m,White-European,no,no,'United Kingdom',no,Parent,YES
7,1,1,1,1,1,1,1,1,0,0,5.0,f,'Middle Eastern ',no,no,Bahrain,no,Parent,YES
8,1,1,1,1,1,1,1,0,0,0,11.0,f,'Middle Eastern ',no,no,Bahrain,no,Parent,YES
9,0,0,1,1,1,0,1,1,0,0,11.0,f,?,no,yes,Austria,no,?,NO


In [9]:
# create input X and output y for training
X = df.drop(['Class/ASD'], axis=1)
y = df['Class/ASD']

In [10]:
X.loc[:10]

,A1_Score,A2_Score,A3_Score,A4_Score,A5_Score,A6_Score,A7_Score,A8_Score,A9_Score,A10_Score,age,gender,ethnicity,jundice,austim,contry_of_res,used_app_before,relation
0,1,1,0,0,1,1,0,1,0,0,6.0,m,Others,no,no,Jordan,no,Parent
1,1,1,0,0,1,1,0,1,0,0,6.0,m,'Middle Eastern ',no,no,Jordan,no,Parent
2,1,1,0,0,0,1,1,1,0,0,6.0,m,?,no,no,Jordan,yes,?
3,0,1,0,0,1,1,0,0,0,1,5.0,f,?,yes,no,Jordan,no,?
4,1,1,1,1,1,1,1,1,1,1,5.0,m,Others,yes,no,'United States',no,Parent
5,0,0,1,0,1,1,0,1,0,1,4.0,m,?,no,yes,Egypt,no,?
6,1,0,1,1,1,1,0,1,0,1,5.0,m,White-European,no,no,'United Kingdom',no,Parent
7,1,1,1,1,1,1,1,1,0,0,5.0,f,'Middle Eastern ',no,no,Bahrain,no,Parent
8,1,1,1,1,1,1,1,0,0,0,11.0,f,'Middle Eastern ',no,no,Bahrain,no,Parent
9,0,0,1,1,1,0,1,1,0,0,11.0,f,?,no,yes,Austria,no,?


In [11]:
X = pd.get_dummies(X)

In [12]:
X.columns.values

array(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score',
       'age_10.0', 'age_11.0', 'age_4.0', 'age_5.0', 'age_6.0', 'age_7.0',
       'age_8.0', 'age_9.0', 'age_nan', 'gender_f', 'gender_m',
       "ethnicity_'Middle Eastern '", "ethnicity_'South Asian'",
       'ethnicity_?', 'ethnicity_Asian', 'ethnicity_Black',
       'ethnicity_Hispanic', 'ethnicity_Latino', 'ethnicity_Others',
       'ethnicity_Pasifika', 'ethnicity_Turkish',
       'ethnicity_White-European', 'jundice_no', 'jundice_yes',
       'austim_no', 'austim_yes', "contry_of_res_'Costa Rica'",
       "contry_of_res_'Isle of Man'", "contry_of_res_'New Zealand'",
       "contry_of_res_'Saudi Arabia'", "contry_of_res_'South Africa'",
       "contry_of_res_'South Korea'",
       "contry_of_res_'U.S. Outlying Islands'",
       "contry_of_res_'United Arab Emirates'",
       "contry_of_res_'United Kingdom'", "contry_of_res_'United States'",
       

In [13]:
# print an example patient from the categorical data
X.loc[1]

A1_Score                               1
A2_Score                               1
A3_Score                               0
A4_Score                               0
A5_Score                               1
A6_Score                               1
A7_Score                               0
A8_Score                               1
A9_Score                               0
A10_Score                              0
age_10.0                               0
age_11.0                               0
age_4.0                                0
age_5.0                                0
age_6.0                                1
age_7.0                                0
age_8.0                                0
age_9.0                                0
age_nan                                0
gender_f                               0
gender_m                               1
ethnicity_'Middle Eastern '            1
ethnicity_'South Asian'                0
ethnicity_?                            0
ethnicity_Asian 

In [15]:
# convert the class data to categorical values (one-hot encoded vectors)
y_categorical = pd.get_dummies(y)

In [16]:
y_categorical.iloc[:10]

,NO,YES
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1
5,1,0
6,0,1
7,0,1
8,0,1
9,1,0


### Split the dataset into training and testing datasets

In [17]:
from sklearn import model_selection
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y_categorical, test_size=0.2)

In [18]:
# examine the shape of the new 4 generated datasets
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(233, 96)
(59, 96)
(233, 2)
(59, 2)


### Build the neural network with Keras

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [22]:
# define the training model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=96, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    
    # compile the model
    adam = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    
    return model

In [23]:
# re-examine the model
model = create_model()
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 776       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 822
Trainable params: 822
Non-trainable params: 0
_________________________________________________________________
None


### Traing the network

In [24]:
model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=1)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
233/233 [==============================] - 0s 1ms/step - loss: 0.6935 - acc: 0.4378
Epoch 2/100
233/233 [==============================] - 0s 81us/step - loss: 0.6928 - acc: 0.5365
Epoch 3/100
233/233 [==============================] - 0s 73us/step - loss: 0.6925 - acc: 0.5408
Epoch 4/100
233/233 [==============================] - 0s 77us/step - loss: 0.6918 - acc: 0.5365
Epoch 5/100
233/233 [==============================] - 0s 77us/step - loss: 0.6898 - acc: 0.5365
Epoch 6/100
233/233 [==============================] - 0s 77us/step - loss: 0.6862 - acc: 0.5365
Epoch 7/100
233/233 [==============================] - 0s 73us/step - loss: 0.6747 - acc: 0.7725
Epoch 8/100
233/233 [==============================] - 0s 77us/step - loss: 0.6470 - acc: 0.8326
Epoch 9/100
233/233 [==============================] - 0s 81us/step - loss: 0.6112 - acc: 0.8670
Epoch 10/100
233/233 [==============================] - 0s 73us/step - loss: 0.5

233/233 [==============================] - 0s 77us/step - loss: 0.0040 - acc: 1.0000
Epoch 83/100
233/233 [==============================] - 0s 68us/step - loss: 0.0038 - acc: 1.0000
Epoch 84/100
233/233 [==============================] - 0s 68us/step - loss: 0.0039 - acc: 1.0000
Epoch 85/100
233/233 [==============================] - 0s 68us/step - loss: 0.0037 - acc: 1.0000
Epoch 86/100
233/233 [==============================] - 0s 68us/step - loss: 0.0034 - acc: 1.0000
Epoch 87/100
233/233 [==============================] - 0s 73us/step - loss: 0.0034 - acc: 1.0000
Epoch 88/100
233/233 [==============================] - 0s 68us/step - loss: 0.0033 - acc: 1.0000
Epoch 89/100
233/233 [==============================] - 0s 68us/step - loss: 0.0033 - acc: 1.0000
Epoch 90/100
233/233 [==============================] - 0s 68us/step - loss: 0.0030 - acc: 1.0000
Epoch 91/100
233/233 [==============================] - 0s 73us/step - loss: 0.0030 - acc: 1.0000
Epoch 92/100
233/233 [===========

### Testing and performance metrics

In [25]:
from sklearn.metrics import classification_report, accuracy_score

predictions = model.predict_classes(X_test)

In [26]:
predictions

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0], dtype=int64)

In [30]:
print("results for categorical model")
print("Accuracy score: {}".format(accuracy_score(y_test[['YES']], predictions)))
print("Classification report:")
print(classification_report(y_test[['YES']], predictions))

results for categorical model
Accuracy score: 0.9830508474576272
Classification report:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        26
           1       1.00      0.97      0.98        33

   micro avg       0.98      0.98      0.98        59
   macro avg       0.98      0.98      0.98        59
weighted avg       0.98      0.98      0.98        59

